# Test Scan merging

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from planeslam.geometry.util import quat_to_rot_mat
from planeslam.general import plot_3D_setup, NED_to_ENU
import planeslam.io as io
from planeslam.extraction import pc_to_planes
from planeslam.scan import pc_to_scan

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

Extract scans from LiDAR point clouds

In [ ]:
# Extract scans
scans = num_scans * [None]

for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])
    scans[i].transform(quat_to_rot_mat(drone_orientations[i,:]), drone_positions[i,:])

In [ ]:
# Plot scans
fig = go.Figure()

for i, scan in enumerate(scans):
    for t in scan.plot_trace():
        fig.add_trace(t)

fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

Test single-scan simplification

In [ ]:
idx = 1
scan = scans[idx]

fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
from planeslam.geometry.plane import BoundedPlane, merge_plane

vertices = list(scan.planes[0].vertices)
update_idxs = []
update_planes = []
vertex_merge_thresh = 2.0

vertices

In [ ]:
# Iterate over remaining planes
for i, p in enumerate(scan.planes[1:]):
    plane_pts = p.vertices
    new_face = -np.ones(4, dtype=int)
    merge_mask = np.zeros(4, dtype=bool)

    # Check if this plane shares any vertices with previous planes
    for k in range(len(vertices)):
        dists = np.linalg.norm(plane_pts - vertices[k], axis=1)
        best_match = np.argsort(dists)[0]
        if dists[best_match] < vertex_merge_thresh:
            new_face[best_match] = k
            merge_mask[best_match] = True

    # If shared, adjust plane accordingly
    if sum(merge_mask) == 2:
        anchor_idxs = new_face[new_face!=-1]
        anchor_verts = np.asarray(vertices)[anchor_idxs]
        new_plane = merge_plane(merge_mask, anchor_verts, plane_pts, p.normal)

        vertices += list(new_plane[~merge_mask,:])
        update_idxs.append(i+1)
        update_planes.append(BoundedPlane(new_plane))
    else:
        vertices += list(plane_pts)

# Update planes
for i, idx in enumerate(update_idxs):
    scan.planes[idx] = update_planes[i]

In [ ]:
update_idxs

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in scan.plot_trace():
    fig.add_trace(t, row=1, col=1)

scan.fuse_edges()

for t in scan.plot_trace():
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

Merge Scans

In [ ]:
merged = scans[0]

for s in scans[1:]:
    merged = merged.merge(s)

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
merged.reduce_inside()
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()